In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from IPython.display import display
import time 
import tracker
latest_date = tracker.processing_dates[-1]
os.chdir('/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [2]:
latest_date

'2022.09.22.15.30'

In [3]:
# setting input and output with jupyter notebook in context 
if 'ipykernel_launcher.py' in sys.argv[0]:
    input_fn = 'results/samplesheets/hicpro/{}.hicpro.samplesheet.with_header.tsv'.format(latest_date)
    output_prefix = 'results/samplesheets/post-hicpro/{}.post-hicpro.post-check.fithichip-peaks'.format(latest_date)
else:
    input_fn = sys.argv[1]
    output_fn = sys.argv[2]

## Helper Functions

In [10]:
def read_out_log(log):
    
    status = 1
    sample_name = 'check'
    read_length = -1
    date = ''

    # extract job_id and index
    meta = os.path.basename(log)
    meta = meta.split('.')[1].split('-')
    job_id = meta[0].replace('o', '')
    index = meta[1]

    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'sample_name' in info:
                sample_name = info.split()[1]
            if 'Using read length' in info:
                read_length = info.split()[4]
            if 'Ended:' in info:
                status = 0

    #return([index, sample_name, read_length, job_id, status])
    return([index, sample_name, job_id, status])

In [11]:
def read_error_log(log):
    
    status = 0
    
    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if '/mnt/beegfs/kfetter' in info:
                status = 2
            if 'WARNING' in info:
                status = 1
    
    return(status)

In [12]:
def read_fithichip_peak_files(file):
    peak_data = {}
    count = 0
    with open(file) as fr:
        for line in fr:
            if line.startswith('c'):
                info = line.strip().split()
                peak_info = info[0:4]
                peak_data[count] = peak_info
                count = count + 1
    return(peak_data)

## Check Logs

In [7]:
# load the data
df = pd.read_table(input_fn)
df['sample_index'] = df.index + 1 # add 1 since we'll be using 1-based indexing with sed

In [8]:
df

,std_sample_name,gse_id,organism,bio_rep,antibody_target,restriction_enzyme,sample_name,sample_index
0,mESC_WT.GSE147919.Mus_Musculus.RYBP.b1,GSE147919,Mus_Musculus,1,RYBP,MboI,mESC_WT,1
1,mESC_WT.GSE147919.Mus_Musculus.RYBP.b2,GSE147919,Mus_Musculus,2,RYBP,MboI,mESC_WT,2


In [9]:
print('number of samples:', len(df))

number of samples: 2


In [17]:
# create a dataframe of log information
output_logs = glob.glob('ref_genome/biorep_merged/results/peaks/logs/run_fithichip_peakcalling.o*')
log_data = []   
for out_log in output_logs:
    sample_index = out_log.split('-')[1]
    #index, sample_name, read_length, job_id, out_status = read_out_log(out_log)
    #print(out_log)
    index, sample_name, job_id, out_status = read_out_log(out_log)
    error_log = [log for log in glob.glob('ref_genome/biorep_merged/results/peaks/logs/run_fithichip_peakcalling.e*') if log.split('-')[1] == sample_index][0]
    error_status = read_error_log(error_log)
    #log_data.append([index, sample_name, read_length, job_id, out_status, error_status, os.path.basename(out_log)])
    log_data.append([index, sample_name, job_id, out_status, error_status, os.path.basename(out_log)])
log_df = pd.DataFrame(log_data)
#log_df.columns = ['sample_index', 'std_sample_name', 'read_length', 'job_id', 'out_status', 'error_status', 'log']
log_df.columns = ['sample_index', 'std_sample_name', 'job_id', 'out_status', 'error_status', 'log']
log_df.drop('sample_index', inplace=True, axis=1)

In [18]:
log_df = log_df.sort_values(by=["std_sample_name"], ascending=True).reset_index(drop=True)
log_df

,std_sample_name,job_id,out_status,error_status,log
0,A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.biorep_merged,6233076,0,0,run_fithichip_peakcalling.o6233076-1
1,A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-2
2,A673-siSA2-Dh8-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-3
3,A673_SA1m1.GSE133227.Homo_Sapiens.CTCF.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-4
4,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-5
5,A673_SA2r.GSE133227.Homo_Sapiens.CTCF.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-6
6,A673_SA2r.GSE133227.Homo_Sapiens.H3K27ac.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-7
7,A673_WT.GSE133227.Homo_Sapiens.CTCF.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-8
8,A673_WT.GSE133227.Homo_Sapiens.H3K27ac.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-9
9,CAR-T-CD19-D10.GSE168881.Homo_Sapiens.H3K27ac.biorep_merged,6233077,0,0,run_fithichip_peakcalling.o6233077-10


## Identifying problem samples

In [19]:
problems = (log_df.out_status != 0) | (log_df.error_status != 0)
problems_df = log_df.loc[problems,:]
print('nubmer of problem samples:', len(problems_df))

nubmer of problem samples: 0


In [20]:
problems_df.loc[:, ['std_sample_name', 'job_id', 'out_status', 'error_status', 'log']]

,std_sample_name,job_id,out_status,error_status,log


In [112]:
ids = []
for i, sample in problems_df.iterrows():
    ids.append(sample["log"].split("-")[1])
print(",".join(ids))

In [17]:
header_output = '{}.with_header.tsv'.format(output_prefix)
problems_df.to_csv(header_output, header=True, index=False, sep='\t')